In [3]:
#pip install wrds

In [ ]:
#https://pypi.org/project/wrds/

In [ ]:
#https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/

In [36]:
import numpy as np
import pandas as pd
import wrds

In [37]:
db = wrds.Connection(wrds_username = 'danielceleny')
#db.create_pgpass_file()

Loading library list...
Done


In [38]:
aapl = db.raw_sql("select date, ret from crsp.dsf where permco in (7) and date >= '1999-12-31'"
                  "and date <='2021-12-31'")
aapl.rename(index = aapl['date'], columns = {'ret':'AAPL'}, inplace = True)
aapl

,date,AAPL
1999-12-31,1999-12-31,0.024922
2000-01-03,2000-01-03,0.088754
2000-01-04,2000-01-04,-0.084310
2000-01-05,2000-01-05,0.014634
2000-01-06,2000-01-06,-0.086538
...,...,...
2021-12-27,2021-12-27,0.022975
2021-12-28,2021-12-28,-0.005767
2021-12-29,2021-12-29,0.000502
2021-12-30,2021-12-30,-0.006578


In [39]:
stocknames = db.get_table(library='crsp', table='stocknames', obs=10)
stocknames

,permno,namedt,nameenddt,shrcd,exchcd,siccd,ncusip,ticker,comnam,shrcls,permco,hexcd,cusip,st_date,end_date,namedum
0,10000.0,1986-01-07,1987-06-11,10.0,3.0,3990.0,68391610,OMFGA,OPTIMUM MANUFACTURING INC,A,7952.0,3.0,68391610,1986-01-07,1987-06-11,2.0
1,10001.0,1986-01-09,1993-11-21,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
2,10001.0,1993-11-22,2008-02-04,11.0,3.0,4920.0,29274A10,EWST,ENERGY WEST INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
3,10001.0,2008-02-05,2009-08-03,11.0,3.0,4920.0,29274A20,EWST,ENERGY WEST INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
4,10001.0,2009-08-04,2009-12-17,11.0,3.0,4920.0,29269V10,EGAS,ENERGY INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
5,10001.0,2009-12-18,2010-07-08,11.0,2.0,4925.0,29269V10,EGAS,ENERGY INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
6,10001.0,2010-07-09,2017-08-03,11.0,2.0,4925.0,36720410,EGAS,GAS NATURAL INC,None,7953.0,2.0,36720410,1986-01-09,2017-08-03,2.0
7,10002.0,1986-01-10,1993-09-29,10.0,3.0,6710.0,60740110,MBNC,MOBILE NATIONAL CORP,None,7954.0,3.0,05978R10,1986-01-10,2013-02-15,2.0
8,10002.0,1993-09-30,1999-06-30,11.0,3.0,6710.0,83623410,SABC,SOUTH ALABAMA BANCORPORATION INC,None,7954.0,3.0,05978R10,1986-01-10,2013-02-15,2.0
9,10002.0,1999-07-01,2002-05-14,11.0,3.0,6020.0,83623410,SABC,SOUTH ALABAMA BANCORPORATION INC,None,7954.0,3.0,05978R10,1986-01-10,2013-02-15,2.0


In [40]:
bonds = db.raw_sql("select caldt, b2ret, b10ret from crsp.mcti where caldt >= '1999-12-31'"
                   "and caldt <= '2021-12-31'", date_cols = ['caldt']).rename(columns = {"caldt": "date"})
bonds

,date,b2ret,b10ret
0,1999-12-31,0.001144,-0.012400
1,2000-01-31,-0.001417,-0.009797
2,2000-02-29,0.006842,0.013598
3,2000-03-31,0.006593,0.034752
4,2000-04-28,0.001496,-0.010026
...,...,...,...
260,2021-08-31,-0.000396,-0.004837
261,2021-09-30,-0.000647,-0.015425
262,2021-10-29,-0.003959,-0.004896
263,2021-11-30,-0.000134,0.012455


In [42]:
#risk free asset
rf = db.raw_sql("select mcaldt, tmytm from crsp.tfz_mth_rf where kytreasnox = 2000001"
                "and mcaldt >= '1978-01-01' and mcaldt <= '2021-12-31'")
rf['tmytm'] = np.exp(rf['tmytm']/12/100)-1
rf.rename(columns = {'mcaldt':'date', 'tmytm':'rf'}, inplace = True)
rf.set_index(['date'], inplace = True)
rf

,rf
date,
1978-01-31,0.005010
1978-02-28,0.005087
1978-03-31,0.005429
1978-04-28,0.005053
1978-05-31,0.005440
...,...
2021-08-31,0.000034
2021-09-30,0.000038
2021-10-29,0.000042


In [41]:
sorted(db.list_libraries())

['aha_sample',
 'ahasamp',
 'auditsmp',
 'auditsmp_all',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compg',
 'compm',
 'compsamp_snapshot',
 'compseg',
 'contrib',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_general',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'contrib_shale',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_indexes',
 'crsp_a_stock',
 'crsp_a_treasuries',
 'crsp_q_mutualfunds',
 'djones',
 'djones_all',
 'dmef',
 'dmef_all',
 'doe',
 'doe_all',
 'etfg_samp',
 'etfgsamp',
 'eurekahedge_sample',
 'eursamp',
 'execcomp',
 'factsamp_all',
 'factsamp_revere',
 'ff',
 'ff_all',
 'fj

In [44]:
db.list_tables(library = 'wrdsapps_finratio')

['firm_ratio', 'id']

In [46]:
db.get_table(library='wrdsapps_finratio', table='firm_ratio', obs=10)

,gvkey,permno,adate,qdate,public_date,capei,be,bm,evm,pe_op_basic,...,ffi30_desc,ffi30,ffi38_desc,ffi38,ffi48_desc,ffi48,ffi49_desc,ffi49,ticker,cusip
0,001000,25881.0,1969-12-31,1970-09-30,1971-01-31,10.949959,10.211,0.427328,1.761483,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,None,None
1,001000,25881.0,1970-12-31,1970-12-31,1971-02-28,35.407426,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
2,001000,25881.0,1970-12-31,1970-12-31,1971-03-31,31.031228,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
3,001000,25881.0,1970-12-31,1970-12-31,1971-04-30,34.213918,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
4,001000,25881.0,1970-12-31,1971-03-31,1971-05-31,11.306738,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
5,001000,25881.0,1970-12-31,1971-03-31,1971-06-30,10.815141,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
6,001000,25881.0,1970-12-31,1971-03-31,1971-07-31,8.029423,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
7,001000,25881.0,1970-12-31,1971-06-30,1971-08-31,7.369151,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
8,001000,25881.0,1970-12-31,1971-06-30,1971-09-30,6.491872,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210
9,001000,25881.0,1970-12-31,1971-06-30,1971-10-31,6.842784,10.544,0.431071,7.009840,None,...,None,None,RUBBR,16.0,RUBBR,15.0,RUBBR,15.0,AE,00003210


In [14]:
db.close()